# Weekly paycheck

Let's first import some tools.

In [ ]:
#import numpy as np
import pandas as pd
import qgrid
import ipywidgets as widgets
#import matplotlib.pyplot as plt

What are your working conditions this week ?

In [ ]:
daily_limit = 8.0  # Daily time limit above it is considered to be overtime
weekly_limit = 40.0  # Weekly hours above which all extra time is considered as overtime
week_rate = 14.50  # Gross salary per hour during the week (Monday to Friday)
weekend_rate = 14.75  # Gross salary per hour during weekend (Saturday and Sunday)
overtime_rate = 22.12  # Gross salary per overtime hour (both week and weekend)
bonus = 0.0  # Bonus salary (tip, etc.)

Initialize the week by running the code box below. Then, update the table to match your own working schedule. Just fill the cell manually using "hh:mm" formatting, that corresponds to hours and minutes. You can select the days you worked by ticking their corresponding boxes in the filtering control menu in the first column of the table.

In [ ]:
days = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
tasks = ["Start", "End", "Break"]
df = pd.DataFrame(index=days, columns=tasks)
df["Start"] = ["08:00" for day in days]
df["End"] = ["16:30" for day in days]
df["Break"] = ["00:30" for day in days]

qgrid_widget = qgrid.show_grid(df)
qgrid_widget

Next, we compute the daily working time of which part may be considered as overtime. Unpaid break times are taken into account by removing their amount from the working time.

In [ ]:
df = qgrid_widget.get_changed_df()
df = df.reindex(index=days, fill_value=pd.NaT)

def str2time(s):
    if pd.isnull(s):
        return pd.NaT
    v = [int(_) for _ in s.split(":")]
    return pd.to_timedelta(v[0], unit="h") + pd.to_timedelta(v[1], unit="m")

dt = df.applymap(str2time)
dt["Working time"] = (dt["End"] - dt["Start"] - dt["Break"]).fillna(pd.to_timedelta(0))
dt["Overtime"] = (dt["Working time"] - pd.to_timedelta(daily_limit, unit="h")).fillna(pd.to_timedelta(0))
dt["Overtime"][dt["Overtime"] < pd.to_timedelta(0)] = pd.to_timedelta(0)

def time2str(t):
    if pd.isnull(t):
        return pd.NaT
    h = t.components.hours
    m = t.components.minutes
    return "{:02d}:{:02d}".format(h, m)

df["Working time"] = dt["Working time"].apply(time2str)
df["Overtime"] = dt["Overtime"].apply(time2str)
df

Moving forward, we compute your total weekly working hours, that we split into regular week time, regular weekend time, and overtime. Overtime can have two origins: either extra working hours above the daily limit, or extra working hours above the weekly limit.

In [ ]:
def time2hour(t):
    if pd.isnull(t):
        return pd.NaT
    return t.components.days * 24 + t.components.hours + t.components.minutes / 60.0

def compute_overtime(week_time, weekend_time, daily_overtime):
    global weekly_limit
    working_time = week_time + weekend_time
    if working_time > weekly_limit:
        overtime = working_time - weekly_limit
    else:
        overtime = daily_overtime
    return overtime

def compute_regular_time(week_time, weekend_time, overtime):
    if week_time >= overtime:
        regular_week_time = week_time - overtime
        regular_weekend_time = weekend_time
    else:
        regular_week_time = 0.0
        regular_weekend_time = weekend_time - (overtime - week_time)
    return regular_week_time, regular_weekend_time

week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
week_time = dt.loc[week]["Working time"].apply(time2hour).sum()
weekend = ["Saturday", "Sunday"]
weekend_time = dt.loc[weekend]["Working time"].apply(time2hour).sum()

daily_overtime = dt["Overtime"].apply(time2hour).sum()
overtime = compute_overtime(week_time, weekend_time, daily_overtime)
regular_week_time, regular_weekend_time = compute_regular_time(
    week_time, weekend_time, overtime)

working_time = dt["Working time"].apply(time2hour).sum()

print("Week time: {:.2f} hours".format(week_time))
print("Weekend time: {:.2f} hours".format(weekend_time))
print("-------------------------------")
print("Regular week time: {:.2f} hours".format(regular_week_time))
print("Regular weekend time: {:.2f} hours".format(regular_weekend_time))
print("Overtime: {:.2f} hours".format(overtime))
print("-------------------------------")
print("Total working time: {:.2f} hours".format(working_time))

Finally, we compute your gross income.

In [ ]:
week_salary = regular_week_time * week_rate
weekend_salary = regular_weekend_time * weekend_rate
overtime_salary = overtime * overtime_rate

salary = week_salary + weekend_salary + overtime_salary + bonus

print("Regular week gross pay: ${:.2f}".format(week_salary))
print("Regular weekend gross pay: ${:.2f}".format(weekend_salary))
print("Overtime gross pay: ${:.2f}".format(overtime_salary))
print("Bonus gross pay: ${:.2f}".format(bonus))
print("-------------------------------")
print("Total gross salary: ${:.2f}".format(salary))

As a example to illustrate `ipywidgets` package, we compute your salary dynamically as a function of your working hours, your pay rates and your bonus salary.

In [ ]:
wt = widgets.FloatSlider(value=week_time, min=0.0, max=80.0)
et = widgets.FloatSlider(value=weekend_time, min=0.0, max=32.0)
dot = widgets.FloatSlider(value=daily_overtime, min=0.0, max=10.0)
wr = widgets.FloatSlider(value=week_rate, min=0.0, max=30.0)
er = widgets.FloatSlider(value=weekend_rate, min=0.0, max=30.0)
_or = widgets.FloatSlider(value=overtime_rate, min=0.0, max=30.0)
b = widgets.FloatSlider(value=bonus, min=0.0, max=200.0)

def printer(wt, et, dot, wr, er, _or, b):
    ot = compute_overtime(wt, et, dot)
    rwt, ret = compute_regular_time(wt, et, ot)
    ws = rwt * wr
    es = ret * er
    os = ot * _or
    s = ws + es + os + b
    print("Regular week time: {:.2f} hours".format(rwt))
    print("Regular weekend time: {:.2f} hours".format(ret))
    print("Overtime: {:.2f} hours".format(ot))
    print("-------------------------------")
    print("Total working time: {:.2f} hours".format(wt + et))
    print("-------------------------------")
    print("Regular week gross salary: ${:.2f}".format(ws))
    print("Regular weekend gross salary: ${:.2f}".format(es))
    print("Overtime gross salary: ${:.2f}".format(os))
    print("Bonus gross pay: ${:.2f}".format(b))
    print("-------------------------------")
    print("Total gross salary: ${:.2f}".format(s))

io = widgets.interactive_output(printer, {
    "wt": wt,
    "et": et,
    "dot": dot,
    "wr": wr,
    "er": er,
    "_or": _or,
    "b": b
})

widgets.HBox([
    widgets.VBox([
        widgets.Label("Week time"),
        widgets.Label("Weekend time"),
        widgets.Label("Daily overtime"),
        widgets.Label("Week rate"),
        widgets.Label("Weekend rate"),
        widgets.Label("Overtime rate"),
        widgets.Label("Bonus")
    ]),
    widgets.VBox([wt, et, dot, wr, er, _or, b]), io
])

In development...

In [ ]:
def compute_salary(week_time, weekend_time, daily_overtime):
    global overtime_rate
    global week_rate
    global weekend_rate

    overtime = compute_overtime(week_time, weekend_time, daily_overtime)
    regular_week_time, regular_weekend_time = compute_regular_time(
        week_time, weekend_time, overtime)

    overtime_salary = overtime * overtime_rate
    week_salary = regular_week_time * week_rate
    weekend_salary = regular_weekend_time * weekend_rate

    return overtime_salary + week_salary + weekend_salary

# print(daily_overtime, overtime, salary)
# plt.figure()
# plt.plot(daily_overtime, salary)
# plt.show()

#plotter(40, 6, 0)

In [ ]:
def f(m, b):
    plt.figure()
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

#interactive_plot = widgets.interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
#output = interactive_plot.children[-1]
#interactive_plot